In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("abebooks.csv")

#### Jak často je Bookbot jediným prodávajícím?

In [3]:
sellers_by_isbn = df.groupby("isbn", as_index=False).agg(
    num_sellers=("seller", "nunique")
)
one_seller_offers = len(sellers_by_isbn[sellers_by_isbn["num_sellers"] == 1])

In [4]:
print(
    f"Bookbot je jediným prodávajícím v {(one_seller_offers/df['isbn'].nunique())*100:.2f}% případů"
)

Bookbot je jediným prodávajícím v 9.60% případů


#### Jak často je cena od Bookbota mezi 25% nejlevnějšími a nejdražšími nabídkami?

In [5]:
price_quantiles = (
    df.groupby("isbn")["price"].quantile([0.25, 0.75]).unstack().reset_index()
)
prices = df[df["seller"] == "Bookbot"].merge(price_quantiles, on="isbn", how="left")

prices["below_25p"] = prices["price"] < prices[0.25]
prices["above_75p"] = prices["price"] > prices[0.75]

In [6]:
print(
    f"Cena od Bookbota je mezi 25% nejlevnějších nabídek v {(prices['below_25p'].sum()/len(prices))*100:.2f}% případů"
)

Cena od Bookbota je mezi 25% nejlevnějších nabídek v 49.40% případů


In [7]:
print(
    f"Cena od Bookbota je mezi 25% nejdražších nabídek v {(prices['above_75p'].sum()/len(prices))*100:.2f}% případů"
)

Cena od Bookbota je mezi 25% nejdražších nabídek v 14.80% případů


#### Identifikujte prodejce, kteří mají častěji nižší cenu než BookBot.

In [8]:
bookbot_prices = df[df["seller"] == "Bookbot"][["isbn", "price"]]
competition_prices = df[df["seller"] != "Bookbot"][["seller", "isbn", "price"]]

In [9]:
price_vs_competition = competition_prices.merge(
    bookbot_prices, on="isbn", suffixes=("", "_bookbot")
)
price_vs_competition["cheaper_than_bookbot"] = (
    price_vs_competition["price"] < price_vs_competition["price_bookbot"]
)
price_vs_competition.groupby("seller", as_index=False).agg(
    cheaper_books=("cheaper_than_bookbot", "sum")
).nlargest(20, "cheaper_books").reset_index(drop=True).rename(
    columns={"seller": "Prodejce", "cheaper_books": "Počet levnějších titulů"}
)

,Prodejce,Počet levnějších titulů
0,medimops,87
1,Book Deals,63
2,moluna,62
3,GreatBookPricesUK,59
4,Books Unplugged,56
5,"GF Books, Inc.",53
6,GreatBookPrices,51
7,Byrd Books,47
8,GoldBooks,45
9,Revaluation Books,42


#### Vypočítejte průměrnou odchylku nabídek Bookotu od nejnižší ceny.

In [10]:
min_price = df.groupby("isbn", as_index=False).agg({"price": "min"})
price_vs_min = df[df["seller"] == "Bookbot"].merge(
    min_price, on="isbn", suffixes=("", "_mininum")
)
price_vs_min["diff"] = price_vs_min["price"] - price_vs_min["price_mininum"]

In [11]:
print(
    f"Průměrná odchylka Bookbota od nejnižší ceny je {price_vs_min['diff'].mean():.2f} EUR"
)

Průměrná odchylka Bookbota od nejnižší ceny je 19.01 EUR


In [12]:
print(
    f"Ale medián odchylky Bookbota od nejnižší ceny je {price_vs_min['diff'].median():.2f} EUR"
)

Ale medián odchylky Bookbota od nejnižší ceny je 3.00 EUR


#### Analyzujte relativní zastoupení nabídek ze zahraničí ve stažených datech.

In [13]:
print(
    f'Podíl zahraničních nabídek v datech je {(len(df[df["seller_country"]!="Deutschland"])/len(df))*100:.2f}%'
)

Podíl zahraničních nabídek v datech je 84.34%
